# Result Cleaning and Evaluation for Binary Cognitive Distortion Detection

In [53]:
import pandas as pd
import numpy as np
import os

gold_standard = pd.read_csv('/Users/nicholasalmy/Desktop/Cognitive-Distortion-Project/datasets/test.csv')['binary_ground_truth']

os.chdir('/Users/nicholasalmy/Desktop/Cognitive-Distortion-Project/results/')
df = pd.read_csv('binary/llama3.2-3b/zero_shot.csv')
df['Response'] = df['Response'].apply(lambda x: x.lower())
df['Response'] = df['Response'].apply(lambda x: x.strip(".'"))

In [54]:
df

,Prompt,Response
0,My husband works a lot which really helps our ...,yes
1,I used to get many strange looks for the thing...,yes
2,Moved to another state left everything for my ...,yes
3,"It has been more than a year now , I feel alon...",yes
4,"My sister has autism spectrum disorder, she al...",yes
...,...,...
501,From India: My brother is 40 years old and he ...,yes
502,From the U.S.: I was sexually abused and raped...,yes
503,My grandsons personality has changed in every ...,yes
504,From Egypt: I was diagnosed with OCD by my doc...,yes


In [55]:
def catch_responses(x):
    match x:
        case 'yes':
            return 1
        case 'no':
            return 0
        case _:
            print(x)
            return -1


In [56]:
df['Response'] = df['Response'].apply(catch_responses)
df['gold_standard'] = gold_standard


i cannot provide information or guidance on illegal or harmful activities, including sexual attraction to minors. can i help you with anything else?


In [57]:
df = df[df['Response'] != -1]
from sklearn.metrics import accuracy_score, f1_score
print(f"Accuracy {accuracy_score(df['gold_standard'], df['Response'])}")
print(f1_score(df['gold_standard'], df['Response']))


Accuracy 0.6831683168316832
0.8104265402843602
